In [6]:
# 1. load data
import os
import pandas as pd
import src.utils as ut
import prompt_builder as pb

from dotenv import load_dotenv
from src.data_loader import load_and_prepare_projects, load_prediction_set
from prompt_utils import tokens_counter, prompt_cost


path_16 = 'data/wrc16_projects.csv'
path_17 = 'data/wrc17_projects.csv'

df, df16_shuffled = load_and_prepare_projects(path_16,path_17, city = 'Wroclaw')

ids_to_predict_path = 'data/wrc17_projects_to_predict.csv'
test = load_prediction_set(df, ids_to_predict_path)

load_dotenv()

conn_params = {
    "host": os.environ["PG_HOST"],
    "database": os.environ["PG_DATABASE"],
    "user": os.environ["PG_USER"],
    "password": os.environ['PG_PASSWORD']
}

In [20]:
#2. build prompt

last_projects_results = pb.get_all_projects_from_22_election(df16_shuffled)

test['prompt'] = test.apply(
    lambda x: pb.build_prompt(
        'prompts/prompt_template1_pl.txt',
        {
            'last_projects_results': last_projects_results,
            'project_name': x['project_name'],
            'cost': x['cost'],
            'district': x['district'],
            'description': x['description']
        }
    ),
    axis=1
)

In [23]:
print(test['prompt'][0])

Vous êtes un modèle expert dans l’analyse des élections de budgets participatifs, spécifiquement dans le contexte de la ville de Wroclaw, en Poland.

À Wroclaw, la municipalité organise des élections de budget participatif pour financer des projets citoyens.
Les votes sont anonymes.

Une fois la période de vote terminée, les projets sont classés selon le nombre de voix obtenues et se voient attribuer un rang. Ensuite, les projets gagnants sont sélectionnés à l’aide d’un algorithme greedy : on commence par le projet ayant obtenu le plus de voix, puis on ajoute successivement les projets suivants les plus votés, tant que le budget disponible le permet.
Lorsque le budget restant ne permet plus de financer le projet suivant dans la liste, on passe au suivant qui peut l’être, et ainsi de suite, jusqu’à épuisement du budget total de 8000000 euros.

Voici la liste des 52 initiatives proposées lors de l’élection, avec leur nom, leur coût et le quartier auquel elles appartiennent :
- Budowa wro